In [29]:
%pip install scikit-learn


  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 7.9 MB/s eta 0:00:00a 0:00:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import cv2
import numpy as np
from scipy.ndimage import label
from sklearn.cluster import DBSCAN


## С шумом

In [23]:
def split_image(image, max_tile_size=(2000, 2000)):
    h, w = image.shape
    if h <= max_tile_size[1] and w <= max_tile_size[0]:
        return [(image, 0, 0)]
    
    tile_size_x = min(w, max_tile_size[0])
    tile_size_y = min(h, max_tile_size[1])
    tiles = []
    
    for y in range(0, h, tile_size_y):
        for x in range(0, w, tile_size_x):
            tile = image[y:y + tile_size_y, x:x + tile_size_x]
            tiles.append((tile, x, y))
    
    return tiles

def detect_objects(image):
    _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    structure = np.ones((3, 3), dtype=np.uint8)
    labeled, num_objects = label(binary, structure)
    
    objects = []
    for i in range(1, num_objects + 1):
        y, x = np.where(labeled == i)
        x_min, x_max = x.min(), x.max()
        y_min, y_max = y.min(), y.max()
        objects.append((x_min, y_min, x_max, y_max))
    
    return objects

def process_image(image_path, max_tile_size=(2000, 2000)):
    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    tiles = split_image(image, max_tile_size)
    all_objects = []
    
    for tile, x_offset, y_offset in tiles:
        objects = detect_objects(tile)
        adjusted_objects = [(x + x_offset, y + y_offset, x_max + x_offset, y_max + y_offset) 
                            for x, y, x_max, y_max in objects]
        all_objects.extend(adjusted_objects)
    
    return all_objects

def draw_objects(image_path, objects, output_path):
    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    for x1, y1, x2, y2 in objects:
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imwrite(output_path, image)

def process_folder(input_folder, output_folder, max_tile_size=(2000, 2000)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_out.bmp")
            
            objects = process_image(input_path, max_tile_size)
            draw_objects(input_path, objects, output_path)
            print(f"Processed: {filename} -> {output_path}")


## Без шума но очень криво

In [25]:

def split_image(image, max_tile_size=(2000, 2000), overlap=100):
    h, w = image.shape
    if h <= max_tile_size[1] and w <= max_tile_size[0]:
        return [(image, 0, 0)]
    
    tile_size_x = min(w, max_tile_size[0])
    tile_size_y = min(h, max_tile_size[1])
    tiles = []
    
    for y in range(0, h, tile_size_y - overlap):
        for x in range(0, w, tile_size_x - overlap):
            tile = image[y:y + tile_size_y, x:x + tile_size_x]
            tiles.append((tile, x, y))
    
    return tiles

def detect_objects(image):
    image = cv2.medianBlur(image, 5)
    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    objects = []
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        objects.append((x, y, x + w, y + h))
    
    return objects

def process_image(image_path, max_tile_size=(2000, 2000)):
    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    tiles = split_image(image, max_tile_size)
    all_objects = []
    
    for tile, x_offset, y_offset in tiles:
        objects = detect_objects(tile)
        adjusted_objects = [(x + x_offset, y + y_offset, x_max + x_offset, y_max + y_offset) 
                            for x, y, x_max, y_max in objects]
        all_objects.extend(adjusted_objects)
    
    return all_objects

def draw_objects(image_path, objects, output_path):
    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    for x1, y1, x2, y2 in objects:
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imwrite(output_path, image)

def process_folder(input_folder, output_folder, max_tile_size=(2000, 2000)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_out.bmp")
            
            objects = process_image(input_path, max_tile_size)
            draw_objects(input_path, objects, output_path)
            print(f"Processed: {filename} -> {output_path}")

## Что то неплохое 

In [2]:

def split_image(image, max_tile_size=(2000, 2000), overlap=100):
    h, w = image.shape
    if h <= max_tile_size[1] and w <= max_tile_size[0]:
        return [(image, 0, 0)]
    
    tile_size_x = min(w, max_tile_size[0])
    tile_size_y = min(h, max_tile_size[1])
    tiles = []
    
    for y in range(0, h, tile_size_y - overlap):
        for x in range(0, w, tile_size_x - overlap):
            tile = image[y:y + tile_size_y, x:x + tile_size_x]
            tiles.append((tile, x, y))
    
    return tiles

def detect_objects(image):
    image = cv2.medianBlur(image, 5)
    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bright_points = []
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bright_points.append((x + w // 2, y + h // 2))
    
    if bright_points:
        bright_points = np.array(bright_points)
        clustering = DBSCAN(eps=20, min_samples=2).fit(bright_points)
        labels = clustering.labels_
        
        clusters = {}
        for i, label in enumerate(labels):
            if label != -1:
                if label not in clusters:
                    clusters[label] = []
                clusters[label].append(bright_points[i])
        
        objects = []
        for cluster in clusters.values():
            cluster = np.array(cluster)
            x_min, y_min = cluster.min(axis=0)
            x_max, y_max = cluster.max(axis=0)
            objects.append((x_min, y_min, x_max, y_max))
        
        return objects
    
    return []

def process_image(image_path, max_tile_size=(2000, 2000)):
    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    tiles = split_image(image, max_tile_size)
    all_objects = []
    
    for tile, x_offset, y_offset in tiles:
        objects = detect_objects(tile)
        adjusted_objects = [(x + x_offset, y + y_offset, x_max + x_offset, y_max + y_offset) 
                            for x, y, x_max, y_max in objects]
        all_objects.extend(adjusted_objects)
    
    return all_objects

def draw_objects(image_path, objects, output_path):
    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    for x1, y1, x2, y2 in objects:
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imwrite(output_path, image)

def process_folder(input_folder, output_folder, max_tile_size=(2000, 2000)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_out.bmp")
            
            objects = process_image(input_path, max_tile_size)
            draw_objects(input_path, objects, output_path)
            print(f"Processed: {filename} -> {output_path}")

In [3]:
input_folder = "images"
output_folder = "output"
process_folder(input_folder, output_folder)

Processed: docks.jpg -> output/docks_out.bmp
Processed: ship.jpg -> output/ship_out.bmp
Processed: river_ships.jpg -> output/river_ships_out.bmp
